---

_Este notebook foi desenvolvido na **version 1.1** ._

---

In [1]:
# Importação das bibliotecas
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Projeto - Teste de hipótese


Definições utilizadas no projeto:

-    Um quartil é um periodo de três meses, onde Q1 é de janeiro a março, Q2 é de abrir até junho, Q3 é de julho atóe setembro e Q4 é de outubro até desembro.

-    Uma recessão será definida com seu início dado por dois quartis consecutivos de queda no GDP e seu fim por dois quartis de aumento do GDP.

-    O periodo basal da recessão é o quartil de menor valor de GDP 

-    Uma cidade universitaria será definida como uma cidade que possui uma porcentagem alta de estudantes em sua população.

O objetivo do projeto é testar a seguinte hipotese:

Cidades universitarias tem o preço de habitações menos afetado por periodos de recessão.

O teste desta hipotese será feito a partir da comparação da razão entre o preço de habitações um quartil antes de uma recessão e durante o periodo basal da recessão utilizando um teste t.

Foram utilizados os seguintes dados para o desenvolvimento do projeto:

* Do site [Zillow research data site](http://www.zillow.com/research/data/) será utilizado o arquivo [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, este arquivo possui preços de venda de casa.

* O site da Wikipedia será utilizado para obter o nome das cidades universitarias. [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) Os dados foram copiados para o arquivo: ```university_towns.txt```.

* Do Bureau of Economic Analysis, US Department of Commerce, o [GDP over time](http://www.bea.gov/national/index.htm#gdp) dos Estados Unidos, em intervalos de quartis, no arquivo ```gdplev.xls```. Serão utilizados apenas dados a partir de 2000.

Os dados utilizados para este codigo devem estar presentes no mesmo ambiente de programação.


In [2]:
# Dicionario utilizado para mapear os nomes de estados para suas siglas
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def get_list_of_university_towns():
    '''
    Retorna um DataFrame de cidades e seus estados obtidos a partir do arquivo university_towns.txt.
    O DataFrame estara no formato DataFrame['State','RegionName']
    
    Será feita a seguinte limpesa nos dados:

    1. Retirada de caracteres com [ no final do campo 'State'
    2. Para o campo 'RegionName', quando aplicavel, deve ser feita a remoção do caracter (
    '''
    
    # Leitura do arquivo .txt direto para DataFrame
    df = pd.read_csv("university_towns.txt",sep='\t', header=None)
    
    df = df[0]
    
    # Busca da substring '[edit]' presente em todos os estados
    sub = '[edit]'    
    df_states = (df[df.str.find(sub)>0]
                 .str.replace("\[.*","")
                 .str.strip())
    
    # Enfatizando que é dataframe
    df = df.to_frame()
    
    # Inicialização da key 'State'
    df['State'] = '0'
    
    # Loop para atribuição dos estados baseado na lista de indices do texto como um todo
    j = 0
    for i in df.index:
        if(j+1<len(df_states.index)):
            if ((df.index[i]>df_states.index[j])&(df.index[i]<df_states.index[j+1])):
                df.loc[i]['State'] = df_states[df_states.index[j]]
            if (df.index[i]>=df_states.index[j+1]):
                j += 1
                df.loc[i]['State'] = df_states[df_states.index[j]]
        else:
            df.loc[i]['State'] = df_states[df_states.index[j]]
    
    # Formatação dos nomes de cidade, foi utilizada uma expressão regex onde:
    # '\(' representa alguma instancia do caracter '('
    # '.' representa qualquer caracter
    # '*' este operador repete a busca anterior até o final da string
    # '\[' representa alguma instancia do caracter '('
    # .str.strip() serve para tirar espaços a direita e a esquerda
    df[0] = (df[0].str.replace("\(.*","")
             .str.replace("\[.*","")
             .str.strip())
    
    # Alterando o nome da coluna de regiões
    df = df.rename(columns = {0:'RegionName'})
    
    # Retirando os estados da lista principla e reiniciando os indices
    # A função drop recebeu df_states.index que é a lista de indices a serem retirados
    df = (df.drop(df_states.index)
          .reset_index(drop=True))
    
    return df[['State','RegionName']]

In [4]:
def get_recession_start():
    '''
    Retorna o ano e o quartil do começo da recessão como um string no
    formato anoquartil (Ex:2004q2)
    '''
    
    colnames = ['Quarter',
               'GDP_Current']
    
    # Leitura e formatação do database de GDP
    df = (pd.read_excel('gdplev.xls',
                       skiprows=5,
                       usecols=[i for i in range(4,6)],
                       headers=True,
                       names = colnames)
                       .drop({0,1}).reset_index(drop=True))
    
    year = '2000q1'
    df = df[df['Quarter'].values.tolist().index('2000q1'):].reset_index(drop=True)
    
    # Loop para encontrar o começo da recessão
    for i in df.index:
        if (i+2>len(df.index)):
            return None
        if ((df.loc[i]['GDP_Current']>df.loc[i+1]['GDP_Current'])&(df.loc[i+1]['GDP_Current']>df.loc[i+2]['GDP_Current'])):
            return df.loc[i]['Quarter']

In [5]:
def get_recession_end():
    '''
    Retorna o ano e o quartil do final da recessão como um string no
    formato anoquartil (Ex:2004q2)
    '''
    
    colnames = ['Quarter',
               'GDP_Current']
    
    # Leitura e formatação do database de GDP
    df = (pd.read_excel('gdplev.xls',
                       skiprows=5,
                       usecols=[i for i in range(4,6)],
                       headers=True,
                       names = colnames)
                       .drop({0,1}).reset_index(drop=True))
    year = '2000q1'
    df = df[df['Quarter'].values.tolist().index('2000q1'):].reset_index(drop=True)
    
    # Obtenção do começo da recessão
    reces_start = get_recession_start()
    
    # Controle para o caso de não ter recessão no periodo
    if(reces_start==None):
        return None
    else:
        df = df[df['Quarter'].values.tolist().index(reces_start):].reset_index(drop=True)
     
    # Loop para encontrar o começo da recessão
    for i in df.index:
        if (i+2>len(df.index)):
            return None
        if ((df.loc[i]['GDP_Current']<df.loc[i+1]['GDP_Current'])&(df.loc[i+1]['GDP_Current']<df.loc[i+2]['GDP_Current'])):
            return df.loc[i+2]['Quarter']

In [6]:
def get_recession_bottom():
    '''
    Retorna o ano e o quartil do periodo basal da recessão como um string no
    formato anoquartil (Ex:2004q2)
    '''
    
    colnames = ['Quarter',
               'GDP_Current']
    
    # Leitura e formatação do database de GDP
    df = (pd.read_excel('gdplev.xls',
                       skiprows=5,
                       usecols=[i for i in range(4,6)],
                       headers=True,
                       names = colnames)
                       .drop({0,1}).reset_index(drop=True))
    year = '2000q1'
    df = (df[df['Quarter'].values.tolist().index('2000q1'):]
          .reset_index(drop=True))
    
    # Obtenção do começo e fim da recessão
    reces_start = get_recession_start()
    reces_end = get_recession_end()
    
    # Controle para o caso de não ter recessão no periodo ou ela ainda não ter terminado
    if((reces_start==None)|(reces_end==None)):
        return None
    else:
        # Obtenção do range de dados para a pesquisa do menor  valor
        df = (df[df['Quarter'].values.tolist().index(reces_start):
                 df['Quarter'].values.tolist().index(reces_end)]
                 .reset_index(drop=True))
        
    # Obtenção do valor minimo de GDP e de seu indice que é passado no df com os quartis
    return df['Quarter'][df['GDP_Current'].idxmin()]

In [7]:
def convert_housing_data_to_quarters():
    '''
    Converte os dados de habitação para quartis e os retorna como valores
    médios no DataFrame. O DataFrame possuí dados com colunas de 2000q1 até
    2016q3 e é multi indexado na forma ["State","RegionName"]
    '''
    
    df = pd.read_csv('City_Zhvi_AllHomes.csv')
    
    # Fazer calculos das médias para cada quadrante
    sub = '2000-01'
    datas = df[df.columns[df.columns.get_loc(sub):]]
    datas.columns = pd.to_datetime(datas.columns)
    datas = datas.resample('1q',how='mean',axis=1)
    
    '''
    Determinar os nomes de quadrantes para as colunas(método alternativo)
    
    colnames = []
    for i in range(0,17):
        if(i<10):
            for j in range(1,5):
                colnames.append('200{}q{}'.format(i,j))
        else:
            for j in range(1,5):
                colnames.append('20{}q{}'.format(i,j))
    colnames.pop(-1)
    '''
    
    # Renomeando as colunas de data utilizando o tipo datetime
    datas = datas.rename(columns=lambda col: '{}q{}'.format(col.year, col.quarter)) 
    
    # Subistituição de sigla para nome de estado e adicionando a coluna ao df
    datas['State'] = df['State'].replace(states)
    
    # Adicionando o campo RegionName ao df
    datas['RegionName'] = df['RegionName']
    
    # Preparando os multiindices
    datas = datas.set_index(['State','RegionName'])
    return  datas

In [78]:
def run_ttest():
    '''
    Essa função prepara os dados e executa o teste t
    '''
    
    # Obtenção da base de dados
    uni_town = get_list_of_university_towns()
    housing = convert_housing_data_to_quarters()
    reces_start = get_recession_start()
    reces_end = get_recession_end()
    reces_bottom = get_recession_bottom()
    
    # Obtenção do range de quartis para a estimativa
    housing = housing[housing.columns[housing.columns.get_loc(reces_start)-1:
                                      housing.columns.get_loc(reces_end)]]
    
    # Obtenção dos dataframes para cidades com universidade e sem universidade
    uni_housing = pd.merge( uni_town, housing, how='inner', left_on=['State','RegionName'], right_index=True).reset_index(drop=True)
    non_uni_housing = pd.merge( uni_town, housing, how='outer', left_on=['State','RegionName'], right_index=True).reset_index(drop=True)
    
    # Expreção para o t-test: price_ratio=quarter_before_recession/recession_bottom
    price_ratio_uni = (uni_housing[uni_housing.columns[uni_housing.columns.get_loc(reces_start)-1]]/
                       uni_housing[uni_housing.columns[uni_housing.columns.get_loc(reces_bottom)]])
    
    price_ratio_non_uni = (non_uni_housing[non_uni_housing.columns[non_uni_housing.columns.get_loc(reces_start)-1]]/
                           non_uni_housing[non_uni_housing.columns[non_uni_housing.columns.get_loc(reces_bottom)]])
    
    # Teste de t de student para verificar a hipotese
    t_test = ttest_ind(price_ratio_uni.dropna(),price_ratio_non_uni.dropna())
    
    if(t_test[1]>0.01):
        return (False, t_test[1],"non-university town")
    else:
        return (True, t_test[1], "university town")

(True, 0.0033861978483629026, 'university town')
